In [51]:
import os
import psycopg2
import importlib
import pandas as pd
import __init__ as ini

importlib.reload(ini)

<module '__init__' from 'e:\\dev\\private-equity\\poc\\notebook\\__init__.py'>

In [52]:
def extract_pnl(file_name):
    hdfcamc_file = os.path.join(ini.raw_data_dir, file_name)
    pnl = pd.read_excel(hdfcamc_file, sheet_name="profit_loss")
    pnl.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

    pnlt = pnl.transpose()                                # Transpose the Table for horizontal format
    pnlt.columns = pnlt.iloc[0]                           # The column name i.e. the mertices are in the first row
    pnlt = pnlt.iloc[1:]                                  # Dropping the first row
    pnlt = pnlt.reset_index()                             # Date is in the index
    pnlt.rename(columns={"index": "Date"}, inplace=True)  # renaming the new columna as Date

    # The date is converted incorrectly from the excel sheet as start of the month instead of end of the month
    # The format of dates in Excel is Mar-14
    pnlt['Date'] = pnlt['Date'] + pd.offsets.MonthEnd(0)  # Setting 0 as parameter ensures that the date does not move to next month incase the date already is end of month

    pnlt.columns = (
        pnlt.columns
        .str.replace('-', '', regex=False)  # 1. Remove all hyphens
        .str.strip()                        # 2. Strip trailing/leading spaces
        .str.replace(' ', '_')              # 3. Replace spaces with underscores
        .str.replace('%', 'percent')        # 4. Replace % with percent
        .str.lower()                        # 5. Lower the column names
    )

    return pnlt

In [53]:
def add_fy_quarter_column(df, date_col='date', new_col='fy'):
    dates = pd.to_datetime(df[date_col])
    
    quarters = pd.Series(index=dates.index, dtype="object")
    fy_years = pd.Series(index=dates.index, dtype="int")

    quarters[dates.dt.month.isin([4, 5, 6])] = 'Q1'
    fy_years[dates.dt.month.isin([4, 5, 6])] = dates.dt.year + 1     # Add 1 year for Indian FY

    quarters[dates.dt.month.isin([7, 8, 9])] = 'Q2'
    fy_years[dates.dt.month.isin([7, 8, 9])] = dates.dt.year + 1     # Add 1 year for Indian FY

    quarters[dates.dt.month.isin([10, 11, 12])] = 'Q3'
    fy_years[dates.dt.month.isin([10, 11, 12])] = dates.dt.year + 1  # Add 1 year for Indian FY

    quarters[dates.dt.month.isin([1, 2, 3])] = 'Q4'
    fy_years[dates.dt.month.isin([1, 2, 3])] = dates.dt.year         # Last Quarter stays in same year

    fy_years = fy_years.astype('Int16')                              # Changing datatype from float to int for easier extraction of year

    df[new_col] = quarters + 'FY' + fy_years.astype(str).str[-2:]    # Extracting last 2 digits from year

    return df

In [54]:
file_name = 'hdfcamc.xlsx'
pnl = extract_pnl(file_name)
company_name = file_name.split('.')[0]

pnl = add_fy_quarter_column(pnl)
pnl.head()


index,date,sales,sales_growth_percent,expenses,manufacturing_cost_percent,employee_cost_percent,other_cost_percent,operating_profit,opm_percent,other_income,...,profit_before_tax,tax_percent,net_profit,exceptional_items_at,profit_excl_excep,profit_for_pe,profit_for_eps,eps_in_rs,dividend_payout_percent,fy
0,2014-03-31,898.0,0.1554,372.0,0.0328,0.1253,0.2566,525.0,0.59,6.0,...,522.0,0.32,358.0,0.0,358.0,358.0,358.0,141.74,0.35,Q4FY14
1,2015-03-31,1056.0,0.1762,432.0,0.0289,0.122,0.2579,624.0,0.59,9.0,...,623.0,0.33,416.0,0.0,416.0,416.0,416.0,164.61,0.39,Q4FY15
2,2016-03-31,1490.0,0.4116,775.0,0.023,0.0971,0.4,715.0,0.48,4.0,...,708.0,0.33,478.0,0.0,478.0,478.0,478.0,189.9,0.42,Q4FY16
3,2017-03-31,1568.0,0.0524,776.0,0.0242,0.1014,0.3693,792.0,0.51,20.0,...,800.0,0.31,550.0,0.0,550.0,550.0,550.0,218.64,0.42,Q4FY17
4,2018-03-31,1870.0,0.1923,802.0,0.1882,0.1005,0.1403,1068.0,0.57,0.0,...,1058.0,0.33,711.0,0.0,711.0,711.0,711.0,33.78,0.47,Q4FY18


In [55]:
print(ini.dsn)

conn = psycopg2.connect(ini.dsn)
df = pd.read_sql_query(f"""
select * from financials.pnl
""", conn)

host=localhost port=5432 dbname=private-equity user=postgres password=asdf


C:\Users\ksstu\AppData\Local\Temp\ipykernel_28040\3894874312.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"""


In [56]:
df

,company_name,fy,sales,sales_growth_percent,expenses,manufacturing_cost_percent,employee_cost_percent,other_cost_percent,operating_profit,opm_percent,...,depreciation,profit_before_tax,tax_percent,net_profit,exceptional_items_at,profit_excl_exceptionals,profit_for_pe,profit_for_eps,eps_in_rs,dividend_payout_percent


In [57]:
df.columns

Index(['company_name', 'fy', 'sales', 'sales_growth_percent', 'expenses',
       'manufacturing_cost_percent', 'employee_cost_percent',
       'other_cost_percent', 'operating_profit', 'opm_percent', 'other_income',
       'exceptional_items', 'other_income_normal', 'interest', 'depreciation',
       'profit_before_tax', 'tax_percent', 'net_profit',
       'exceptional_items_at', 'profit_excl_exceptionals', 'profit_for_pe',
       'profit_for_eps', 'eps_in_rs', 'dividend_payout_percent'],
      dtype='object')

In [58]:
pnl.columns

Index(['date', 'sales', 'sales_growth_percent', 'expenses',
       'manufacturing_cost_percent', 'employee_cost_percent',
       'other_cost_percent', 'operating_profit', 'opm_percent', 'other_income',
       'exceptional_items', 'other_income_normal', 'interest', 'depreciation',
       'profit_before_tax', 'tax_percent', 'net_profit',
       'exceptional_items_at', 'profit_excl_excep', 'profit_for_pe',
       'profit_for_eps', 'eps_in_rs', 'dividend_payout_percent', 'fy'],
      dtype='object', name='index')